<img align="right" src="images/etcbc.png"/>

# MQL versus TF-Query

See [tfVersusMql](tfVersusMql.ipynb) for an introduction.

# Loading

We load the Text-Fabric program and the BHSA data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use
from tf.core.helpers import project

from util import getTfVerses, getShebanqData, compareResults, MQL_RESULTS

In [3]:
VERSION = '2017'
# A = use('bhsa', hoist=globals(), version=VERSION)
A = use('bhsa:clone', checkout="clone", hoist=globals(), version=VERSION)

# Example 10a

[Bas Meeuse: Example 10: NTN + object](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4416)

```
// The object follows the clause-initial predicate
[clause
  [phrase first function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object follows the predicate
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The clause-initial object precedes the predicate
[clause
  [phrase FOCUS first function = Objc]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object precedes the predicate
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The clause-initial object is a pronominal suffix
[clause
  [phrase first function = PreO
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object is a pronominal suffix
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function = PreO
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]


```

In [4]:
(verses, words) = getShebanqData(A, MQL_RESULTS, "10a")

232 results in 190 verses with 458 words


First we grasp the structure of this query.

All alternatives have, in one way or another

* **phraseNTN** a predicate phrase with the verb `NTN[` in the `qal`
* **phraseNTNps** a predicate phrase with the verb `NTN[` in the `qal` and a pronominal suffix
* **phrases0** a string of zero or more phrases whose function is not `Objc` and not `Cmpl`
* **phrases1** a string of one or more phrases whose function is not `Objc` and not `Cmpl`
* **phraseObj** an object phrase
* a condition that states that after matching the previous elements, there are no
  more phrases with function `Objc` or `Cmpl`

Now let us state the alternatives in pseudo TF-query terms.
We leave out the condition for now:

**A**

```
clause
  =: phraseNTN
  <: phrases0
  <: phraseObj
```

**B**

```
clause
  =: phrases1
  <: phraseNTN
  <: phrases0
  <: phraseObj
```

**C**

```
clause
  =: phraseObj
  <: phrases0
  <: phraseNTN
```

**D**

```
clause
  =: phrases1
  <: phraseObj
  <: phrases0
  <: phraseNTN
```

**E**

```
clause
  =: phraseNTNps
```

**F**

```
clause
  =: phrases1
  <: phraseNTNps
```
  

**N.B. The results that follow E and F do not have a FOCUS block, so nothing of these results
will show up in SHEBANQ.**

While that is probably not what is meant, when we compute the words and verses of the results in TF,
we will leave these out as well.

We can combine **A** and **B** and we can do the same with **C** and **D**, and also **E** and **F**:

**AB**

```
clause
  =: phrases0
  <: phraseNTN
  <: phrases0
  <: phraseObj
```

**CD**

```
clause
  =: phrases0
  <: phraseObj
  <: phrases0
  <: phraseNTN
```

**EF**

```
clause
  =: phrases0
  <: phraseNTNps
```

We cannot make a faithful TF-Query out of this, because of the `*` in the block:

```
[phrase not function IN (Objc, Cmpl)] *
```

What we do is: we make a very simple query for clauses
with a **phraseNTN** and a **phraseObj**,
and for each of them we test whether its phrases conform to one
of the patterns **A** .. **F** above.

In [5]:
query1 = """
clause
  phrase function=Pred|PreS
    word vs=qal lex=NTN[
  phrase function=Objc
"""

query2 = """
clause
  phrase function=PreO
    word vs=qal lex=NTN[
"""

In [6]:
resultCandidates1 = A.search(query1)
resultCandidates2 = A.search(query2)

  0.89s 1246 results
  0.63s 243 results


We merge the two candidate sets, but the second set has tuples without a member for the object phrase.
We put in `-1` there.
Later, when we display the tuples, the `-1` does no harm.

In [7]:
resultCandidates2a = [(*r, -1) for r in resultCandidates2]
resultCandidates = sorted(resultCandidates1 + resultCandidates2a)

In [8]:
def getPhrases(cl):
    phrases = L.d(cl, otype="phrase")
    maxPos = 0
    consecutivePhrases = []
    for phrase in phrases:
        words = L.d(phrase, otype="word")
        start = words[0]
        end = words[-1]
        # exclude phrases inside gaps
        if start < maxPos:
            continue
        consecutivePhrases.append(phrase)
        maxPos = end
    return tuple(consecutivePhrases)
    
results = []

objlike = {"Objc", "Cmpl"}

for r in resultCandidates:
    cl = r[0]
    ntn = r[1]
    obj = r[3]
    phrases = getPhrases(cl)
    
    predSeen = False
    objSeen = False
    
    good = True
    
    for p in getPhrases(cl):
        fn = F.function.v(p)
            
        if not predSeen and not objSeen:
            if p == ntn:
                predSeen = True
            elif p == obj:
                objSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif predSeen and not objSeen:
            if p == obj:
                objSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif not predSeen and objSeen:
            if p == ntn:
                predSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif predSeen and objSeen:
            if fn in objlike:
                good = False
                break
            else:
                continue
                
    if not good:
        continue
    
    if not predSeen:
        continue
        
    pfn = F.function.v(ntn)
    
    if objSeen:
        if pfn == "PreO":
            continue
    else:
        if pfn != "PreO":
            continue
            
    results.append((cl, ntn, obj))
    
print(f"{len(results)} results")

232 results


In [9]:
(tfVerses, tfWords) = getTfVerses(A, results, (2,))

190 verses
458 words


In [10]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [11]:
A.table(results, end=5)

n,p,clause,phrase,phrase
1,Genesis 15:10,וַיִּתֵּ֥ן אִישׁ־בִּתְרֹ֖ו,יִּתֵּ֥ן,אִישׁ־בִּתְרֹ֖ו
2,,עַל־כֵּ֥ן לֹא־נְתַתִּ֖יךָ,נְתַתִּ֖יךָ,
3,Genesis 23:13,נָתַ֜תִּי כֶּ֤סֶף הַשָּׂדֶה֙,נָתַ֜תִּי,כֶּ֤סֶף הַשָּׂדֶה֙
4,Genesis 30:18,נָתַ֤ן אֱלֹהִים֙ שְׂכָרִ֔י,נָתַ֤ן,שְׂכָרִ֔י
5,Genesis 30:26,תְּנָ֞ה אֶת־נָשַׁ֣י וְאֶת־יְלָדַ֗י,תְּנָ֞ה,אֶת־נָשַׁ֣י וְאֶת־יְלָדַ֗י


Or even closer:

In [12]:
A.show(results, end=5, condenseType="clause")

**Conclusion**

This was not an easy one to do by hand-coding.

But the good thing is that once we have obtained the results, we can inspect them really closely.